In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from datetime import date

import matplotlib.pyplot as plt
%matplotlib inline
import sys
from sklearn.feature_extraction import DictVectorizer

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy

from xgboost import XGBRegressor

In [2]:
num_f = ['model_age', 'm_from_car_reg', 'power_kw', 'mileage_in_km']
cat_f = ['brand', 'model', 'color', 'transmission_type', 'fuel_type']
target = ['price_in_euro']
preprocessed_data = '../data/car-price-processed-data.parquet'

In [3]:
#loading data
df = pd.read_parquet(preprocessed_data)

In [4]:
df.dtypes

brand                 object
model                 object
color                 object
transmission_type     object
fuel_type             object
model_age              int64
m_from_car_reg         int64
power_kw               int64
mileage_in_km        float64
price_in_euro          int64
dtype: object

In [5]:
df

,brand,model,color,transmission_type,fuel_type,model_age,m_from_car_reg,power_kw,mileage_in_km,price_in_euro
0,alfa-romeo,Alfa Romeo GTV,red,Manual,Petrol,28,333,148,160500.0,1300
1,alfa-romeo,Alfa Romeo 164,black,Manual,Petrol,28,341,191,190000.0,24900
2,alfa-romeo,Alfa Romeo Spider,black,Unknown,Petrol,28,341,110,129000.0,5900
3,alfa-romeo,Alfa Romeo Spider,black,Manual,Petrol,28,336,110,189500.0,4900
4,alfa-romeo,Alfa Romeo 164,red,Manual,Petrol,27,320,132,96127.0,17950
...,...,...,...,...,...,...,...,...,...,...
251074,volvo,Volvo XC40,white,Automatic,Hybrid,0,3,192,1229.0,57990
251075,volvo,Volvo XC90,white,Automatic,Diesel,0,4,173,4900.0,89690
251076,volvo,Volvo V60,white,Automatic,Diesel,0,2,145,1531.0,61521
251077,volvo,Volvo XC40,white,Automatic,Hybrid,0,2,132,1500.0,57890


In [17]:
X = df[cat_f].values[:,0:5]
encoded_x = None

for i in range(0, X.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(X[:,i])
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = numpy.concatenate((encoded_x, feature), axis=1)
print("X shape: : ", encoded_x.shape)

X shape: :  (244342, 1336)


In [18]:
encoded_x
encoded_x.shape

(244342, 1336)

In [19]:
X = df[num_f].values[:,0:4]
encoded_x = numpy.concatenate((encoded_x, X), axis=1)

In [20]:
encoded_x
encoded_x.shape

(244342, 1340)

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(encoded_x, df[target].values, test_size=0.25, random_state=0)

In [24]:
Y_train

array([[ 8600],
       [24025],
       [48490],
       ...,
       [19950],
       [17990],
       [ 2500]])

In [25]:
# fit model on training data
model = XGBRegressor()
model.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [26]:
Y_train

array([[ 8600],
       [24025],
       [48490],
       ...,
       [19950],
       [17990],
       [ 2500]])

In [27]:
print(model)

from sklearn.metrics import accuracy_score

# make predictions for test data
Y_pred = model.predict(X_train)
# evaluate predictions
Y_pred = [round(value) for value in Y_pred]
RSME = metrics.mean_squared_error(Y_train, Y_pred, squared = False)
print("RSME train: ", RSME)

# make predictions for test data
Y_pred = model.predict(X_test)
# evaluate predictions
Y_pred = [round(value) for value in Y_pred]
RSME = metrics.mean_squared_error(Y_test, Y_pred, squared = False)
print("RSME test: ", RSME)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)
RSME train:  4948.784289824164
RSME test:  5250.046269504856


In [40]:
Y_pred

[28990,
 21829,
 5650,
 11503,
 10093,
 32496,
 20823,
 55522,
 4406,
 25794,
 23752,
 26211,
 23209,
 52197,
 10698,
 24897,
 44752,
 12263,
 10060,
 29124,
 34672,
 12541,
 34019,
 18011,
 4035,
 18502,
 8830,
 30424,
 10426,
 21508,
 33674,
 27028,
 10787,
 15058,
 19419,
 23598,
 63924,
 17306,
 18559,
 30639,
 50856,
 13250,
 5894,
 17107,
 18833,
 7373,
 10789,
 15772,
 16337,
 12454,
 18399,
 13377,
 10184,
 5826,
 30045,
 21941,
 34578,
 35808,
 7418,
 9882,
 15712,
 74990,
 37474,
 20083,
 12790,
 37961,
 30045,
 1533,
 9366,
 17610,
 27431,
 66417,
 34235,
 7330,
 5857,
 17464,
 6086,
 45066,
 7910,
 24331,
 15211,
 21351,
 17480,
 29963,
 13283,
 9886,
 7797,
 12871,
 18262,
 20331,
 47403,
 18812,
 8726,
 20814,
 31832,
 12715,
 30965,
 12296,
 16136,
 28398,
 55120,
 33046,
 35845,
 29151,
 6216,
 28246,
 29861,
 51963,
 15913,
 28047,
 14090,
 12534,
 15868,
 29610,
 75092,
 6530,
 32712,
 6167,
 40859,
 8598,
 23200,
 34630,
 62676,
 27361,
 47803,
 17299,
 36699,
 16854

In [29]:
Y_test

array([[59999],
       [18950],
       [ 5990],
       ...,
       [ 5999],
       [ 3950],
       [23449]])

In [30]:
import pickle
with open('../models/xgb_regressor.bin', 'wb') as f_out:
    pickle.dump(model,f_out)